In [ ]:
#mounting google drive
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [9]:
import pandas as pd
#opening the data file in read mode
df = pd.read_csv('train.csv', sep=',')
print(df)


                                                  review sentiment
0      SAPS AT SEA <br /><br />Aspect ratio: 1.37:1<b...  negative
1      If you want mindless action, hot chicks and a ...  positive
2      "The Woman in Black" is easily one of the cree...  positive
3      I can barely find the words to describe how mu...  negative
4      What's in here ?! Let me tell you. It's the pr...  negative
...                                                  ...       ...
29995  I was really looking forward to this show give...  negative
29996  I searched for this movie for years, apparentl...  positive
29997  This is a story of the Winchester Rifle Model ...  positive
29998  this film is in the MANDINGO & DRUM type<br />...  negative
29999  Ha ha. - oh no - what to say about this film? ...  negative

[30000 rows x 2 columns]


In [27]:
reviews = df.iloc[:, 0].values  # Access first column (review)
labels = df.iloc[:, 1].values   # Access second column (sentiment)


In [28]:
#parsing html
from bs4 import BeautifulSoup
import re

def parseHtml(html):
  soup = BeautifulSoup(html, 'html.parser')
  return soup.get_text()

def removeDigits(string):
  for i in range(10):
    string=string.replace(str(i),' ')
  return string

#removing html
reviews=list(map(parseHtml, reviews))

#removing digits
reviews=list(map(removeDigits, reviews))


C:\Users\mashh\AppData\Local\Temp\ipykernel_14300\920268986.py:6: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(html, 'html.parser')


In [29]:
#tokenizing
import nltk
nltk.download('punkt')
tokenizedText=[nltk.word_tokenize(item) for item in reviews]


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\mashh\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [30]:
#removing punctuation
punc = '''!()-[]{};:'"\, <>./?@#$%^&*_~'''
tokenizedText= [[word for word in review if word not in punc] for review in tokenizedText]


In [23]:
import numpy as np
np.shape()

ValueError: invalid literal for int() with base 10: 'SAPS'

In [31]:
import numpy as np
#splitting the Dataset into train and test set



  # Convert to a NumPy array
 # Access the shape correctly
totalRows = len(tokenizedText)
splitRatio=0.75
splitPoint=int(splitRatio*totalRows)

trainReviews=tokenizedText[:splitPoint]
trainLabels=labels[:splitPoint]
testReviews=tokenizedText[splitPoint:]
testLabels=labels[splitPoint:]


In [32]:
#learning word embeddings on training data using Gensim library
+
++

In [41]:
def getVectors(dataset):
  singleDataItemEmbedding=np.zeros(embeddingsSize)
  vectors=[]
  for dataItem in dataset:
    wordCount=0
    for word in dataItem:
      if word in model.wv.key_to_index:
        singleDataItemEmbedding=singleDataItemEmbedding+model.wv[word]
        wordCount=wordCount+1
  
    singleDataItemEmbedding=singleDataItemEmbedding/wordCount  
    vectors.append(singleDataItemEmbedding)
  return vectors

trainReviewVectors=getVectors(trainReviews)
testReviewVectors=getVectors(testReviews)

In [42]:
#Let's define a function that can display the accuracy, F1-score, label-wise precision, recall, etc. of each classifier

from sklearn.metrics import accuracy_score
#add path of google drive to environment variable to load python files from google drive
from sklearn.metrics import precision_recall_fscore_support as score
from sklearn.metrics import f1_score

def printResults(y_true, y_predicted):
  print("Accuracy= ", accuracy_score(y_true, y_predicted))

  columns=['false', 'true']

  precision, recall, fscore, support = score(y_true, y_predicted)

  print('###########################################')
  print('precision: {}'.format(precision))  
  print('recall: {}'.format(recall))
  print('fscore: {}'.format(fscore))
  print('support: {}'.format(support))
  print('###########################################3')

  print('Macro F1 ',f1_score(y_true, y_predicted, average='macro'))

  print('Micro F1 ', f1_score(y_true, y_predicted, average='micro'))



In [43]:
#naive bayes
from sklearn.preprocessing import MinMaxScaler
from sklearn.naive_bayes import MultinomialNB
clfNB = MultinomialNB()

scaler = MinMaxScaler()
scaledTrainX= scaler.fit_transform(trainReviewVectors)
scaledTestX = scaler.fit_transform(testReviewVectors)
clfNB.fit(scaledTrainX, trainLabels)

#test naive bayes accuracy
testLabelsPredicted=list(clfNB.predict(scaledTestX))

#print results
print("####################RESULTS OF NAIVE BAYES CLASSIFIER##################")
printResults(testLabelsPredicted, testLabels)


####################RESULTS OF NAIVE BAYES CLASSIFIER##################
Accuracy=  0.6169333333333333
###########################################
precision: [0.51776246 0.71727468]
recall: [0.64948454 0.59514801]
fscore: [0.57619118 0.65052913]
support: [3007 4493]
###########################################3
Macro F1  0.6133601556744168
Micro F1  0.6169333333333333


In [44]:
#neural network
from sklearn.neural_network import MLPClassifier

clfMLP = MLPClassifier(hidden_layer_sizes=(10, 10, 10))
clfMLP.fit(trainReviewVectors, trainLabels)
  
testLabelsPredicted=list(clfMLP.predict(testReviewVectors))

#print results
print("####################RESULTS OF NEURAL NETWORK CLASSIFIER##################")
printResults(testLabelsPredicted, testLabels)

####################RESULTS OF NEURAL NETWORK CLASSIFIER##################
Accuracy=  0.8254666666666667
###########################################
precision: [0.84544008 0.80525751]
recall: [0.81455939 0.83737796]
fscore: [0.8297125  0.82100369]
support: [3915 3585]
###########################################3
Macro F1  0.825358096840683
Micro F1  0.8254666666666667


In [45]:
# Random Forest Classifier
from sklearn.ensemble import RandomForestClassifier

clfRF = RandomForestClassifier(n_estimators = 1000)
# Train the model on training data
clfRF.fit(trainReviewVectors, trainLabels);

testLabelsPredicted=list(clfRF.predict(testReviewVectors))

#print results
print("####################RESULTS OF Random Forest CLASSIFIER##################")
printResults(testLabelsPredicted, testLabels)

####################RESULTS OF Random Forest CLASSIFIER##################
Accuracy=  0.7854666666666666
###########################################
precision: [0.76723224 0.80391631]
recall: [0.79834483 0.77341935]
fscore: [0.78247938 0.78837301]
support: [3625 3875]
###########################################3
Macro F1  0.7854261970937771
Micro F1  0.7854666666666666


In [46]:
#KNN classifier
from sklearn.neighbors import KNeighborsClassifier

clfKNN = KNeighborsClassifier(n_neighbors=3)

# Train the model on training data
clfKNN.fit(trainReviewVectors, trainLabels);

testLabelsPredicted=list(clfKNN.predict(testReviewVectors))

#print results
print("####################RESULTS OF KNN Classifier##################")
printResults(testLabelsPredicted, testLabels)

####################RESULTS OF KNN Classifier##################
Accuracy=  0.6964
###########################################
precision: [0.71818664 0.67435622]
recall: [0.69054295 0.7028236 ]
fscore: [0.70409357 0.68829569]
support: [3923 3577]
###########################################3
Macro F1  0.6961946275682361
Micro F1  0.6964
